In [1]:
from aerolib import *
from xfoillib import *
import pandas as pd
import numpy as np

In [2]:
def isNaN(num):
    return num != num

In [3]:
class InsufficientInputData(Exception):
    def __init__(self, text):
        self.txt = text

In [4]:
class Foil():
    def __init__(self, name):
        pass
    def get_geometry(self):
        pass
    def get_filename(self):
        pass
    

In [21]:
class Sculptor():
    def __init__(self, performance_file_name, params_file_name, settings_file_name, m):
        self.params_file_name = params_file_name
        self.geom_file_name = "GEOM.csv"
        df = pd.read_csv(performance_file_name, header=None)[1:]
        self.performance = dict(zip(df[0], [float(x) for x in df[1]]))
        self.preprocess_performance()

        df = pd.read_csv(settings_file_name, header=None)[1:]
        self.settings = dict(zip(df[0], [x for x in df[1]]))
        self.preprocess_settings()

        df=pd.read_csv(params_file_name, index_col=False)
        self.params = dict(filter(lambda x: x[0][0]!="#", dict(zip(df["NAME"], [float(x) for x in df["VALUE"]])).items()))
        self.preprocess_params()

        self.geom = {}
        self.aero = {"cruise":np.nan}
        self.tom = m

    def preprocess_params(self):
        critical_params = ['CL_take_off', 'eta_prop', 'A_aft', 
                           'B_keel', 'l_stab']
        for name in critical_params:
            if isNaN(self.params.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    def preprocess_performance(self):
        critical_performance = ['cruise_speed', 'take_off_speed', 'flight_time']
        for name in critical_performance:
            if isNaN(self.performance.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')
    
    def preprocess_settings(self):
        if isNaN(self.settings.get('g', np.nan)):
            self.settings['g'] = 9.81
            
        if isNaN(self.settings.get('density', np.nan)):
            self.settings['density'] = 1.22

        if isNaN(self.settings.get('wire_scale_coef', np.nan)):
            self.performance['wire_scale_coef'] = 1.3
        
        critical_settings = ['dyn_viscosity', 'density', 'g', 'Re', 'M', 
                             'xfoil_max_it', 'ncr', 'alpha_min', 'alpha_max', 'alpha_step', 
                             'ar_min', 'ar_max', 'ar_delta', 'osvald_coef', 
                             'wire_scale_coef', 'XFoil_path', 'foil1_name', 'work_dir']
        for name in critical_settings:
            if isNaN(self.settings.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    def calculate_geometry(self):        
        self.geom["wing_area"] = wing_area(self.tom, float(self.settings["g"]),
                                           float(self.settings["density"]), 
                                           float(self.performance["take_off_speed"]), 
                                           float(self.params["CL_take_off"]))
        self.CL_cr = CL_cruise(self.tom, self.performance["cruise_speed"],
                               float(self.geom["wing_area"]), float(self.settings["g"]), 
                               float(self.settings["density"]))
        ar_step_number = int(abs(float(self.settings["ar_max"]) - float(self.settings["ar_min"])) // float(self.settings["ar_delta"]))
        
        ar_range = np.linspace(float(self.settings["ar_min"]), float(self.settings["ar_max"]), ar_step_number)
        
        self.geom["AR"] = AR_selector(ar_range, self.geom, 
                                      self.settings, self.performance, self.tom)
        self.aero["cruise"] = K_V_solver(self.geom, self.settings, 
                                         self.performance["cruise_speed"], 
                                         self.geom["AR"], self.tom)
        self.geom["ba"] = ba(self.geom["AR"], self.geom["wing_area"])
        self.geom["wingspan"] = wingspan(self.geom["AR"], self.geom["wing_area"])
        self.geom["aft_area"] = aft_area(float(self.params["A_aft"]), 
                                         self.geom["wing_area"], self.geom["ba"], 
                                         float(self.params["l_stab"]))
        self.geom["keel_area"] = keel_area(float(self.params["B_keel"]), 
                                           self.geom["wing_area"], self.geom["wingspan"], 
                                           float(self.params["l_stab"]))
        self.geom["V_dihedral"] = gamma(self.geom["keel_area"], self.geom["aft_area"])
        self.geom["Vtail_area"] = stab_area(self.geom["aft_area"], self.geom["V_dihedral"])
        self.geom["P_cruise"] = P_cruise(self.tom, self.aero["cruise"].K[0], 
                                        float(self.params["eta_prop"]), float(self.settings["g"]))
        self.geom["wire_length"] = wire_length(self.geom["wingspan"], float(self.params["l_stab"]), 
                                              float(self.settings["wire_scale_coef"]))

        self.geom["V_dihedral"] = math.degrees(gamma(self.geom["keel_area"], self.geom["aft_area"]))
        return 0
        
    def update_m(self, new_m):
        self.tom = new_m
    
    def write_geometry(self):
        with open(self.geom_file_name, 'w') as f:
            f.write("%s, %s\n" % ("NAME", "VALUE"))
            for key in self.geom.keys():
                f.write("%s, %s\n" % (key, self.geom[key]))

    def get_data_to_weigh(self):
        return [self.params_file_name, self.geom_file_name]

In [22]:
'''  
Ожидаемый формат params-csv-файла
NAME  INDEX  VALUE CONJUGATE
0   m1    1      1            NaN
1   n1    1     10            NaN
2   m2    2      2            NaN
3   n2    2    100            NaN
4   m3    NaN      3            NaN
5    m    NaN   1000             cy

Ожидаемый формат geomerty-csv-файла
NAME      VALUE
wing_area 1
ba        1
wingspan  1
AR        1
'''

#m (масса ЛА) -- есть сумма
#слагаемые бывают трёх типов:
#1) произведение величин из params с одинаковым индексом
#2) произведение двух величин -- одна из params, одна из geometry (её имя - CONJUGATE)
#3) величина из params
def weigh(params_file_name, geometry_file_name, ask_about_no_value=True):
    geom_df = pd.read_csv(geometry_file_name, header=None).loc[1:] #удаление заголовка из DF
    geom = dict(zip(geom_df[0], geom_df[1])) #создание словаря {Cy: <val>, ...}

    #создание таблицы с колонками вида 
    #[<colomn_num>, <NAME>, <INDEX>, <VALUE>, <CONJUGATE>]
    params = pd.read_csv(params_file_name, index_col=False) 
    params = pd.DataFrame(filter(lambda x: x["NAME"][0] != "#", [params.iloc[i] for i in range(len(params.axes[0]))])).T
    
    #создание словаря 
    #{<значение колонки index из params>: <список значений всех параметров с этим индексом>}
    multiply_dict = dict()
    multiply_dict_names = dict()
    m = 0
    
    for var in params: #итерация по колонкам; в var попадает номер колонки (colomn_num)
        params[var]["VALUE"] = float(params[var]["VALUE"])
        if isNaN(params[var]["VALUE"]):
            if ask_about_no_value:
                ans = input(f"\nУ ПАРАМЕТРА {params[var]['NAME']} нет значения - пропустить (y/n)?")
            else:
                ans = "y"
            if ans == "y":
                continue
            raise InsufficientInputData(f"no value for variable '{params[var]['NAME']}'")

        if not isNaN(params[var]["INDEX"]): #если тип (1)
            #создать в multiply_dict ключ с найденым индексом,
            #со значением пустого списка, если такого ключа ещё нет
            multiply_dict[params[var]["INDEX"]] = multiply_dict.get(params[var]["INDEX"], [])
            multiply_dict_names[params[var]["INDEX"]] = multiply_dict_names.get(params[var]["INDEX"], [])
            #добавить значение этого параметра
            #в список индекса найденного индекса
            multiply_dict[params[var]["INDEX"]].append(params[var]["VALUE"])
            multiply_dict_names[params[var]["INDEX"]].append(params[var]["NAME"])
        
        elif not isNaN(params[var]["CONJUGATE"]): #если тип (2)
            conjugates_list = "".join(params[var]["CONJUGATE"].split()).split(",")
            try:
                conjugates = list(map(lambda conj: 1/float(geom[conj[2:]]) if conj[0:2] == "1/" else float(geom[conj]), conjugates_list))
            except KeyError as e:
                print(e)
                raise InsufficientInputData(f"for variable '{params[var]['NAME']}', '{params[var]['CONJUGATE']}' "+\
                                                f"is specified as conjugate, but it is no '{params[var]['CONJUGATE']}'"+\
                                                f" in the geom-file")
            for conj in conjugates:
                m += conj * params[var]["VALUE"]
            print(f"+ {params[var]['NAME']}*", "*".join(conjugates_list), " : ",  m, sep="")
            

    #вычисление слагаемых типа (1) -- перемножение значения параметров с одинаковыми индексами
    for item in multiply_dict.items():
        key, vals = item
        m += math.prod(vals)
        print("+", "*".join(multiply_dict_names[key]) , ":",  m)
    
    print(f"------------------------\nИтоговая масса: {m}\n------------------------\n\n")
    return m

In [23]:
# Модуль расчёта геометрических характеристик потребляет на вход: 
# dataframe PERFORMANCE с потребными эксплуатационными характеристиками
# dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
# пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
# TOM - значение взлётной массы в начальном приближении.

# PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

# PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
# line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
# area_dens_LWPLA
# energy_dens_bat
# number_servo1, number_servo2
# l_stab

# Все единицы в СИ
def inner_iteration():
    performance_file_name = "PERFORMANCE.csv"#input("имя файла performances: ")
    params_file_name = "PARAMS.csv" #input("имя файла params: ")
    settings_file_name = "SETTINGS.csv" #input("имя файла settings: ") 
    tom = 2 #input("нулевое приближение влётной массы: ")
    max_iter = 10 #input("максимальное число итераций: ")
    
    df = pd.read_csv(settings_file_name, header=None)
    tom_eps = float(dict(zip(df[0], df[1]))["tom_eps"])
    
    sculptor = Sculptor(performance_file_name, params_file_name, settings_file_name, tom)
    i = 1
    ask_about_no_value = True
    while True:
        sculptor.calculate_geometry()
        sculptor.write_geometry()
        pf, gf = sculptor.get_data_to_weigh()

        print(f"итерация {i}")
        new_tom = weigh(pf, gf, ask_about_no_value)
        ask_about_no_value = False
        if abs(tom - new_tom) > tom_eps:
            tom = new_tom
            sculptor.update_m(new_tom)
        else:
            print(f"сошлось на итерации: {i}")
            print("информация сохранена в файл с геометрией")
            break
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                print("последняя итерация геометрии сохранена")
                break
            
        i+=1
    

In [26]:
inner_iteration()

на одном из удлинений XFoil выдал отбивку
на одном из удлинений XFoil выдал отбивку
на одном из удлинений XFoil выдал отбивку
итерация 1



У ПАРАМЕТРА capacity_LiIon18650 нет значения - пропустить (y/n)? y


+ specific_wing*wing_area : 0.3207229928541404
+ specific_aft*Vtail_area : 0.575762071163366
+ specific_3wire*wire_length : 0.6213581092831856



У ПАРАМЕТРА mPito нет значения - пропустить (y/n)? y


+ l_stab*tube6X5 : 0.6291581092831856
+ mVTX*nVTX : 0.6531581092831856
+ mCrossfire*nCrossfire : 0.6581581092831856
+ mRunc_s4*nRunc_s4 : 0.6681581092831856
+ mReg*nReg : 0.7181581092831857
+ mAT2308*nAT2308 : 0.7761581092831857
+ mLiIon18650*nLiIon18650 : 0.9171581092831858
+ mLiIon21700*nLiIon21700 : 0.9171581092831858
+ mfus*nfus : 1.0671581092831857
+ mMG90*nMG90 : 1.0671581092831857
+ mES9051*nES9051 : 1.0911581092831857
+ mPixhawk*nPixhawk : 1.1391581092831857
+ nPito : 1.1391581092831857
------------------------
Итоговая масса: 1.1391581092831857
------------------------


на одном из удлинений XFoil выдал отбивку
итерация 2
+ specific_wing*wing_area : 0.18267709907168364
+ specific_aft*Vtail_area : 0.24250354129104398
+ specific_3wire*wire_length : 0.2637135390534997
+ l_stab*tube6X5 : 0.2715135390534997
+ mVTX*nVTX : 0.2955135390534997
+ mCrossfire*nCrossfire : 0.3005135390534997
+ mRunc_s4*nRunc_s4 : 0.3105135390534997
+ mReg*nReg : 0.3605135390534997
+ mAT2308*nAT2308 : 0.41